# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

## Face Detection vs. Face Recognition

In [1]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255 -> 0 ~ 1 (1 불투명, 0 투명)
    
    for c in range(0, 3): # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))
        # 투명한 영역은 대상 이미지, 완전 불투명한 영역은 오버레이 이미지 사용


# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 모듈
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 모듈

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 영상 저장
fourcc = cv2.VideoWriter_fourcc(*'DIVX') # 'D', 'I', 'V', 'X'
width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('face_output.avi', fourcc, fps, (width, height))

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100
image_right_eye = cv2.resize(image_right_eye, (100, 100))
image_left_eye = cv2.resize(image_left_eye, (100, 100))
image_nose = cv2.resize(image_nose, (300, 100))

# model_selection : 0은 가까운 거리, 1은 5미터 이내 / min_detection_confidence : 신뢰도
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.7
) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)
        
        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections: # 검출된 얼굴이 있으면 그림
            # 6개 특징 : 좌우 눈, 코 끝, 입, 좌우 귀(귀구슬점)
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye, left_eye, nose_tip = keypoints[0], keypoints[1], keypoints[2]
                
                h, w, _ = image.shape # hegiht, width, channel
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
                # cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA)
                
                # 각 특징에 이미지 그리기
                # image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                # image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                # image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = image_nose
                
                # 캐릭터 그리기
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 150, 50, image_nose)
        
        # Flip the image horizontally for a selfie-view display.
        out.write(image)
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break

out.release()
cap.release()
cv2.destroyAllWindows()